<div align="center">

# Heart Attack Extensive EDA, Visualizations and Prediction
</div>

About this dataset Age : Tuổi của bệnh nhân

Sex : Giới tính của bệnh nhân  (1 = male; 0 = female)

ca : số lượng các mạch chính (0-3)

cp : Loại đau ngực

Value 1: đau thắt ngực điển hình
Value 2: đau thắt ngực không điển hình
Value 3: không đau thắt ngực
Value 4: Không có triệu chứng

trestbps - huyết áp lúc nghỉ (tính bằng mm Hg khi nhập viện)

chol : cholestoral in mg/dl được tính bởi cảm biến BMI

thalach - nhịp tim tối đa đạt được

fbs : (Lượng đường trong máu > 120 mg/dl) (1 = true; 0 = false)

rest_ecg : kết quả điện tâm đồ

Value 0: Bình thường Value 1: Có sóng bất thường (Sóng T bị đảo ngược and/or ST vênh hướng lên trên or xuống of > 0.05 mV) Value 2: hiển thị có thể xảy ra or phì đại thất trái( nhịp tim tối đa đạt được)

target : 0= Nguy cơ bị đau tim thấp 1= Nguy cơ bị đau tim cao


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.metrics import accuracy_score, precision_score, classification_report, plot_confusion_matrix, roc_curve, auc, confusion_matrix

import warnings 
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
data = pd.read_csv("../input/heart-attack-analysis-prediction-dataset/heart.csv")
data.head()

## Exploring the Data

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

## Exploratory Data Analysis

### Checking the Data Distribution

In [ ]:
val_counts = data["output"].value_counts()
no_heart_attack = (val_counts[0] / data.shape[0]) * 100
heart_attack = (val_counts[1] / data.shape[0]) * 100

print(f"Heart Attack: {math.floor(heart_attack)}%")
print(f"No Heart Attack: {math.ceil(no_heart_attack)}%")

print()

sns.barplot(x = ["No Heart Attack", "Heart Attack"], y = [no_heart_attack, heart_attack])
plt.show()


Phần trăm Đau tim là 54 và Phần trăm Không Đau tim là 46. Vì vậy, tập dữ liệu được cân bằng và không cần phải cân bằng dữ liệu

## Bivariate Analysis

### Plots for columns with categorical values

<b>
Kiểm tra sự phân bố giới tính đối với bệnh tim</b>

In [ ]:
ax = plt.figure(figsize = (9,5))
sns.countplot(
    data=data,
    x = "sex",
    hue="output",
)
plt.title("Distribution of Gender w.r.t Heart Attack")
plt.show()

In [ ]:
val_counts = data.groupby("sex")["output"].value_counts()
male_heart_attack_percentage = (val_counts[1][1] / (val_counts[1][1]+val_counts[1][0])) * 100
female_heart_attack_percentage = (val_counts[0][1] / (val_counts[0][1]+val_counts[0][0])) * 100

print(f"Male heart Attack: {male_heart_attack_percentage}%")
print(f"Female heart Attack: {female_heart_attack_percentage}%")

In [ ]:
plt.figure(figsize = (9,5))
x = ["Male","Female"]
y = [male_heart_attack_percentage,female_heart_attack_percentage]
plt.barh(x, y)
  
for index, value in enumerate(y):
    plt.text(value, index,
             str(value)[:4]+"%")

plt.title("Gender vs Heart Attack")
plt.show()


Tỷ lệ Đau tim ở nữ là 75% và ở nam là khoảng 45%, tức là nữ có nhiều cơ hội bị Đau tim hơn.

<b>Kiểm tra sự phân bố của Đau thắt ngực do tập thể dục (exng) liên quan đến Bệnh tim</b>

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(
    data=data,
    x = "exng",
    hue="output",
)
plt.title("Distribution of Exercise Induced Angina w.r.t Heart Attack")
plt.show()

In [ ]:
exang_val_counts = data.groupby("exng")["output"].value_counts()
exang_heart_attack_percentage = (exang_val_counts[1][1] / (exang_val_counts[1][1]+exang_val_counts[1][0])) * 100
no_exang_heart_attack_percentage = (exang_val_counts[0][1] / (exang_val_counts[0][1]+exang_val_counts[0][0])) * 100

print(f"Exang heart Attack: {exang_heart_attack_percentage}%")
print(f"No Exang heart Attack: {no_exang_heart_attack_percentage}%")

plt.figure(figsize = (9,5))
x = ["Exang","No Exang"]
y = [exang_heart_attack_percentage,no_exang_heart_attack_percentage]
plt.barh(x, y)
  
for index, value in enumerate(y):
    plt.text(value, index,
             str(value)[:4]+"%")

plt.title("Exercise Induced Angina vs Heart Attack")
plt.show()

Từ dữ liệu trên, chúng ta có thể thấy rằng những người bị Đau thắt ngực do tập thể dục có tỷ lệ đau tim là 23,2% . Trong khi những người không bị Đau thắt ngực do tập thể dục có tỷ lệ đau tim là 69,6%. Do đó những người bị Đau thắt ngực do tập thể dục ít có nguy cơ bị đau tim hơn.

<b>
Kiểm tra sự phân bố của Đau ngực liên quan đến Bệnh tim</b>

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(
    data=data,
    x = "cp",
    hue="output",
)
plt.title("Distribution of Chest Pain w.r.t Heart Attack")
plt.show()

In [ ]:
cp_val_counts = data.groupby("cp")["output"].value_counts()

x = []
y = []

for i in range(4):
    temp_per = (cp_val_counts[i][1] / (cp_val_counts[i][1]+cp_val_counts[i][0])) * 100
    x.append("CP_"+str(i))
    y.append(temp_per)
    print(f"CP_{i} Heart Attack Percentage: {temp_per}%")
    
print()

plt.figure(figsize = (9,5))
plt.barh(x[::-1], y[::-1])
  
for index, value in enumerate(y[::-1]):
    plt.text(value, index,
             str(value)[:4]+"%")

plt.title("Chest Pain vs Heart Attack")
plt.show()

<ul>
    <li>Value 0: typical angina</li>
    <li>Value 1: atypical angina</li>
    <li>Value 2: non-anginal pain</li>
    <li>Value 3: asymptomatic</li>
</ul>
<br/>
Những người bị đau ngực loại 1 (đau thắt ngực không điển hình) dễ bị đau tim hơn so với các loại đau ngực khác.

<b>Kiểm tra sự phân phối của Đường trong máu liên quan đến Bệnh tim</b>

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(
    data=data,
    x = "fbs",
    hue="output",
)
plt.title("Distribution of Fasting Blood Sugar w.r.t Heart Attack")
plt.show()

In [ ]:
fbs_val_counts = data.groupby("fbs")["output"].value_counts()
fbs_heart_attack_percentage = (fbs_val_counts[1][1] / (fbs_val_counts[1][1]+fbs_val_counts[1][0])) * 100
no_fbs_exang_heart_attack_percentage = (fbs_val_counts[0][1] / (fbs_val_counts[0][1]+fbs_val_counts[0][0])) * 100

print(f"FBS heart Attack: {exang_heart_attack_percentage}%")
print(f"No FBS heart Attack: {no_exang_heart_attack_percentage}%")

print()

plt.figure(figsize = (9,5))
x = ["FBS","No FBS"]
y = [fbs_heart_attack_percentage,no_fbs_exang_heart_attack_percentage]
plt.barh(x, y)
  
for index, value in enumerate(y):
    plt.text(value, index,
             str(value)[:4]+"%")

plt.title("Fasting Blood Sugar(FBS) vs Heart Attack")
plt.show()


**FSB : Lượng đường trong máu**

Không có nhiều sự khác biệt giữa những người có FBS và những người không có FBS. Có một chút khác biệt là 4,9%

<b>
Kiểm tra sự phân bố kết quả điện tâm đồ lúc nghỉ liên quan đến Bệnh tim</b>

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(
    data=data,
    x = "restecg",
    hue="output",
)
plt.title("Distribution of Resting Electrocardiographic Results w.r.t Heart Attack")
plt.show()

restecg_val_counts = data.groupby("restecg")["output"].value_counts()

x = []
y = []

for i in range(3):
    temp_per = (restecg_val_counts[i][1] / (restecg_val_counts[i][1]+restecg_val_counts[i][0])) * 100
    x.append("RESTECG_"+str(i))
    y.append(temp_per)
    print(f"RESTECG_{i} Heart Attack Percentage: {temp_per}%")
    
print()

plt.figure(figsize = (9,5))
plt.barh(x[::-1], y[::-1])
  
for index, value in enumerate(y[::-1]):
    plt.text(value, index,
             str(value)[:4]+"%")

plt.title("Resting Electrocardiographic Results vs Heart Attack")
plt.show()

<ul>
    <li>RESTECG_0: Bình thường</li>
    <li>RESTECG_1: có bất thường sóng ST-T (đảo ngược sóng T và / hoặc ST chênh lên hoặc giảm> 0,05 mV)</li>
    <li>RESTECG_2:hiển thị phì đại thất trái có thể xảy ra hoặc xác định theo tiêu chí của Estes</li>
</ul> Những người bị RESTECG 1 dễ bị đau tim hơn do bị RESTECG 0 và RESTECG 2

### Đồ thị cho các cột có giá trị Liên tục

<b>Kiểm tra sự phân bố của Tuổi đối với Bệnh tim</b>

In [ ]:
ax = plt.figure(figsize = (9,5))
sns.histplot(
    data=data,
    x = "age",
    hue="output",
    binwidth=2
)
plt.title("Distribution of Age w.r.t Heart Attack")
plt.show()


Những người từ 50-70 tuổi dễ bị Đau tim hơn.

In [ ]:
continuous_var_cols = ["age", "trtbps", "chol", "thalachh", "oldpeak", "output"]

sns.pairplot(data[continuous_var_cols], hue="output")
plt.show()


Trong biểu đồ trên, chúng ta có thể thấy rằng không có hai cột nào có mối tương quan cao. Do đó, không cần loại bỏ tính năng đối với các cột có dữ liệu liên tục.

<b>Biểu đồ phân phối cho tất cả các cột có các biến liên tục</b>

In [ ]:
for col in continuous_var_cols[:-1]:
    plt.figure(figsize = (9,5))
    sns.histplot(
        data=data,
        x = col,
        hue="output",
        binwidth=2,
        kde=True
    )
    plt.title(f"Distribution of {col.upper()} w.r.t Heart Attack")
    plt.show()
    print()

<b>Kiểm tra mối tương quan giữa các thuộc tính</b>

In [ ]:
corr_matrix = data[continuous_var_cols].corr()

plt.figure(figsize=(11,7))
sns.heatmap(corr_matrix, center=0, annot=True)
plt.title("Correlation among Attributes")
plt.show()

## Xây dựng mô hình dự đoán

In [ ]:
#split the data to feature and label

X = data.iloc[:, 0: -1]
y = data.iloc[:, -1:]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 0)

In [ ]:
#scalled the data
scaler = StandardScaler()

X_test_scaled = scaler.fit_transform(X_test)
X_train_scaled = scaler.fit_transform(X_train)

y_train_scaled = scaler.fit_transform(y_train)
y_test_scaled = scaler.fit_transform(y_test)
print(f"Train Data: {X_train.shape}, {y_train.shape}")
print(f"Test Data: {X_test.shape}, {y_test.shape}")
print('X_test: {}'.format(X_test.head(25)))
print('Y_test: {}'.format(y_test.head(25)))

In [ ]:
#scale
knn = KNeighborsClassifier(n_neighbors = 7)  
knn.fit(X_train_scaled, y_train)

knn_predicted = knn.predict(X_test_scaled)

#model fit check 
train_score = knn.score(X_train_scaled, y_train)
test_score = knn.score(X_test_scaled, y_test)
print('test fit score: {} %'.format(knn.score(X_test_scaled, y_test)*100))
print('train fit score: {} %'.format(knn.score(X_train_scaled, y_train)*100))
# print('Confusion matrix: {}'.format(confusion_matrix(y_test, knn_predicted)))
print('Accuracy score: {} %'.format(accuracy_score(y_test, knn_predicted)*100))
# print('X_test: {}'.format(X_test.head(25)))
# print('Y_test: {}'.format(y_test.head(25)))
print(knn_predicted)

In [ ]:
data.head(300)

# **Example bệnh nhân vào viện :**

Tuổi  : 60 , age = 60

Giới tính : Nam , sex = 1

Kiểu đau ngực : Không bị đau ngực , cp = 3

Huyết ấp lúc nghỉ ngơi : 145 , trtbps = 145

Chol : 176 , chol = 176

Đường huyết lúc đói : 200 , fbs = 1

Kết quả điện tâm đồ lúc nghỉ : bình thường , restecg = 0

Nhịp tim tối đa đạt được : 90 , thalachh = 90

Đau thắt ngực do tập thể dục : Không , exng = 0


Old peak : Dốc lên , oldpeak = 1.0****

slp : 1

Độ dốc của đoạn tập luyện : Dốc lên, caa = 2

thal : 1

**==> [60,1,3,145,176,1,0,90,0,1.0,1,2,1]**


In [ ]:
# thall can change result
test = pd.DataFrame({
    "age" : [60],
    "sex": [1],
    "cp" : [3],
    "trtbps": [145],
    "chol": [176],
    "fbs":[1],
    "restecg":[0],
    "thalachh": [90],
    "exng": [0],
    "oldpeak": [1.0],
    "slp": [1],
    "caa" : [2],
    "thall" :[1]
})
test = test.append(X_test,ignore_index = True)
test_scaler = scaler.fit_transform(test)
predicted= knn.predict(test_scaler)
print(test.iloc[0])
print('kết quả: ',predicted[0])

In [ ]:
test = pd.DataFrame({
    "age" : [41],
    "sex": [0],
    "cp" : [1],
    "trtbps": [130],
    "chol": [204],
    "fbs":[0],
    "restecg":[0],
    "thalachh": [172],
    "exng": [0],
    "oldpeak": [1.4],
    "slp": [2],
    "caa" : [0],
    "thall" :[2]
})
test = test.append(X_test,ignore_index = True)
test_scaler = scaler.fit_transform(test)
predicted= knn.predict(test_scaler)
print(test.iloc[0])
print('kết quả: ',predicted[0])

## Conclusion
extensive exploratory data analysis

Sau khi phân tích dữ liệu khám phá rộng rãi (EDA) và thử nghiệm với nhiều mô hình phân loại, những nhận xét này là:

1. Tỷ lệ Đau tim ở nữ là 75% và ở nam là khoảng 45%, tức là nữ có nhiều cơ hội bị Đau tim hơn.
2. Những người bị Đau thắt ngực do tập thể dục có tỷ lệ đau tim là 23,2% trong khi những người không bị Đau thắt ngực do tập thể dục có tỷ lệ đau tim là 69,6%. Do đó những người bị Đau thắt ngực do tập thể dục ít có nguy cơ bị đau tim hơn.
3. Những người bị đau ngực loại 1 (đau thắt ngực không điển hình) dễ bị đau tim hơn.
4. Những người bị RESTECG 1 dễ bị đau tim hơn do RESTECG 0 và RESTECG 2 gây ra.
5. Những người ít tuổi có khả năng bị Đau tim cao hơn.
6. Những người từ 50-70 tuổi dễ bị Đau tim hơn.

<table>
    <tr>
        <th>Metrics</th>
        <th>KNN</th>
    </tr>
    <tr>
        <th>Accuracy</th>
        <td>80%</td>
    </tr>
    <tr>
        <th>Precision</th>
        <td>0.86</td>
    </tr>
</table>
<br/>
